# Multi-Agent Poem Generation

Jupyter Notebook for build a Multi-Agent poem generator using Langchain & LangGraph.

Prerequisites:

1. Python (v3.8.1 or above)
2. Ollama (v0.3.9 or above)
    1. Moondream 2 model
    2. Gemma 2: 2B model


In [64]:
%%capture --no-stderr
%pip install -U langgraph langsmith
%pip install -U langchain_ollama

In [65]:
import os

os.environ["LANGCHAIN_PROJECT"] = "Multi-Agent Poem Generation"

## Helper functions
Here, we define two helper functions for creating LLM agents and LangChain nodes that can be added to the graph and utilize message history.

In [66]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama


def create_agent(llm, system_prompt):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    return prompt | llm

In [67]:
from langchain_core.messages import HumanMessage


def agent_node(state, agent, name):
    result = agent.invoke(state["messages"])
    return {"messages": [HumanMessage(content=result.content, name=name)]}

## Creating the graph
One of the central concepts of LangGraph is state. Each graph execution creates a state that is passed between nodes in the graph as they execute, and each node updates this internal state with its return value after it executes. 

Here, we create a simple State class that defines how messages between agents are stored within the State. We then create a StateGraph object, which is the graph in which all agents are added to.

In [68]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

Next, we can create the agents and provide each with distinct prompts, ensuring that each agent exhibits unique behaviors within the graph.

In [69]:
analysis_prompt = """You are the Analyst Agent, an expert in visual analysis. 
Your task is to examine images in detail and provide a comprehensive description 
of what you see. Describe the scene as thoroughly as possible"""

creative_prompt = """You are the Creator Agent, a talented poet and creative writer. 
Your task is to take detailed descriptions of images provided by the Analyst Agent 
and craft a funny poem based on the imagery and emotions conveyed. """

analysis_llm = ChatOllama(model="moondream")
creative_llm = ChatOllama(model="gemma2:2b")

analysis_agent = create_agent(analysis_llm, creative_prompt)
creative_agent = create_agent(creative_llm, creative_prompt)

After creating the agents, we can then create nodes from them, and define how they are connecting up to each other within the graph.

In [70]:
import functools

analysis_node = functools.partial(
    agent_node, agent=analysis_agent, name="Analysis Agent"
)
creative_node = functools.partial(
    agent_node, agent=creative_agent, name="Creative Agent"
)

graph_builder.add_node("Analysis Agent", analysis_node)
graph_builder.add_node("Creative Agent", creative_node)

graph_builder.add_edge(START, "Analysis Agent")
graph_builder.add_edge("Analysis Agent", "Creative Agent")
graph_builder.add_edge("Creative Agent", END)

graph = graph_builder.compile()

## Passing in multi-modal data
LangGraph supports Multi-Modal LLMs, allowing us to create an initial message for the graph that includes both images and text. To achieve this, we first convert the image into a Base64-encoded binary string.


In [71]:
import base64

image_file = "hedgehog.jpg"
image_path = f"images/{image_file}"

with open(image_path, "rb") as file:
    encoded_string = base64.b64encode(file.read()).decode("utf-8")

message = HumanMessage(
    content=[
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{encoded_string}"},
        },
    ],
)

Finally, we can run the graph with our message as the initial input using the `stream()` method.

In [72]:
for event in graph.stream({"messages": [message]}):
    for value in event.values():
        message = value["messages"][-1]
        print(f"---\n{message.name}: {message.content}")

---
Analysis Agent: 
The image features a small white and brown hedgehog sitting in the grass. The hedgehog is facing towards the camera, giving us a close-up view of its adorable appearance. It appears to be enjoying itself as it lounges on the green grassy field.
---
Creative Agent: A tiny prince, with quills so neat, 
In emerald fields, a joyful treat!
He yawns, he smiles, a prickly grin,
His furry face, where mischief spins.

His nose is twitching, keen and bright,
For juicy worms, a pure delight! 
A fluffy crown upon his head,
Of softest fur, of purest dread.

He dreams of berries, plump and red,
And feasts in secret, in the grass he's tread.
Oh, humble hedgehogs, small and bold,
With stories to tell, yet to be told! 




